In [3]:
from openai import AzureOpenAI
import time

**FidChat**

In [7]:
instructions = (
    "És o **FIDCHAT**, um assistente virtual especializado em apoio a agentes de vendas da Fidelidade. "
    "A tua função é prestar esclarecimentos exclusivamente sobre os produtos da Fidelidade My Savings e PPR Evoluir, "
    "com base integral e rigorosa na documentação oficial que te foi fornecida. Podes também fazer comparações com produtos de competidores que estejam presentes na documentação.\n\n"

    "### Limites de atuação:\n"
    "- Responde **sempre com base na documentação**.\n"
    "- **Nunca inventes informações (não alucines).**\n"
    "- Caso sejas questionado sobre outros temas, informa que a tua função se limita ao apoio sobre a Fidelidade, os seus produtos financeiros e a comparação com produtos de competidores.\n\n"

    "### Estilo e linguagem:\n"
    "- Responde **sempre na mesma língua que a mensagem do utilizador.**\n"
    "  - Ex: se o utilizador escrever em inglês, responde em inglês.\n"
    "  - Ex: se escrever em português, responde em português de Portugal.\n"
    "  - Ex: se escrever em espanhol, responde em espanhol.\n"
    "- **Nunca mistures línguas na mesma resposta.**\n"
    "- **Mantém o mesmo nível de formalidade usado pelo utilizador.**\n"
    "  - Se o utilizador tratar por \"tu\", responde também por \"tu\".\n"
    "  - Se o utilizador usar tratamento mais formal (como \"você\" ou linguagem mais profissional), mantém o mesmo grau de formalidade.\n"
    "- Sê claro, objetivo e profissional, mas mantém um tom cordial e acessível.\n"
    "- Utiliza formatação Markdown quando útil (ex: listas, negrito, subtítulos, tabelas).\n\n"

    "### Cumprimentos e expressões sociais:\n"
    "- Sempre que o utilizador disser algo como obrigado, obrigada, olá, bom dia, boa tarde ou expressar gratidão ou cumprimento, responde de forma educada e simpática.\n"
)


In [8]:
# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint="https://ai-bcds.openai.azure.com/",
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
    api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o-mini", 
    instructions = instructions,
    tool_resources={"code_interpreter":{"file_ids":["assistant-WZQHoFwobKHT1ZWKkEAGqm","assistant-MsQgxjsadXVcZKLw2fJwNh","assistant-LpaoUAR5Rq3MgXfMSU4i9A","assistant-KW358eNBGny1N5DM8LoBaC","assistant-CiKqiv2zmbtF8FH2iQAKAw","assistant-9yhV7cKx5ynoLqoPbs1B67","assistant-5QanyuMMkjkkfVUv6rkCfM"]}},
    temperature=0.2,
    top_p=1
    )

In [13]:
import time
import re

def chat_with_assistant(client, assistant_id, message, instructions):
    # Create a new thread
    thread = client.beta.threads.create()

    # Add message (with optional system instructions)
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=instructions + "\n\n" + message
    )

    # Run assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Wait until response is ready
    while run.status in ["queued", "in_progress", "cancelling"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Return the assistant's response
    if run.status == "completed":
        messages_list = client.beta.threads.messages.list(thread_id=thread.id)
        for msg in reversed(messages_list.data):
            if msg.role == "assistant":
                raw_response = msg.content[0].text.value
                clean_response = re.sub(r"【\d+:\d+†source】", "", raw_response).strip()
                return clean_response
    return "[ERROR] Failed to get assistant response."


In [14]:
message = 'hey'
chat_with_assistant(client, assistant.id, message, instructions = instructions)

'Olá! Como posso ajudar-te hoje com informações sobre os produtos da Fidelidade My Savings ou PPR Evoluir?'

**EvalChat**

In [19]:
client2 = AzureOpenAI(
  azure_endpoint = "https://ai-bcds.openai.azure.com/",
  api_key= "8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
  api_version="2024-05-01-preview"
)

assistant2 = client.beta.assistants.create(
  model="gpt-4o-mini-BCwDS", 
  instructions="",
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_ezpssNVZDpwb0RjgcxSwbNZz"]}},
  temperature=1,
  top_p=1
)

In [24]:
def eval_chat(client, assistant_id, question, answer, instructions=None):
    import time
    import re

    if instructions is None:
        instructions = (
            "You are an evaluation assistant. Given a user question and a proposed answer, "
            "your task is to assess the **accuracy, completeness, and relevance** of the answer. "
            "You must compare it against the documents you have access to. "
            "If the answer is incorrect, incomplete, or misleading, explain why. "
            "The output should be a similarity score and a summary of improvements\n\n"
        )

    # Construct the input message
    eval_prompt = (
        f"{instructions}"
        f"### Question:\n{question}\n\n"
        f"### Proposed Answer:\n{answer}"
    )

    # Create a new thread
    thread = client.beta.threads.create()

    # Add message to thread
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=eval_prompt
    )

    # Run assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Wait until response is ready
    while run.status in ["queued", "in_progress", "cancelling"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Return assistant's evaluation
    if run.status == "completed":
        messages_list = client.beta.threads.messages.list(thread_id=thread.id)
        for msg in reversed(messages_list.data):
            if msg.role == "assistant":
                raw_response = msg.content[0].text.value
                clean_response = re.sub(r"【\d+:\d+†source】", "", raw_response).strip()
                return clean_response
    return "[ERROR] Failed to get assistant response."


chatfit

In [16]:
message = 'O Fidelidade Savings é mais indicado para que perfil de risco?'
response = chat_with_assistant(client, assistant.id, message, instructions = instructions)
response

'O **Fidelidade My Savings** é um produto de poupança que se destina a investidores com um perfil de risco **baixo a moderado**. Este produto é ideal para aqueles que procuram uma forma segura de poupança, com um retorno estável e previsível, sem expor o capital a grandes oscilações de mercado.\n\nSe tiveres mais perguntas sobre este produto ou outros da Fidelidade, estou à disposição para ajudar!'

In [ ]:
instructions2 = "You are an evaluation bot. please compare the answers\
    given as a message to the files you have access to, and output a similarity score.\
    you will receive a message with the format: \
    - question \
    - answer \
    You must look for the question in the files you have access to and compare both answers"


eval

In [26]:
question = 'O Fidelidade Savings é mais indicado para que perfil de risco?'
message2 = 'O **Fidelidade My Savings** é um produto de poupança que se destina \
            a investidores com um perfil de risco **baixo a moderado**. Este produto \
            é ideal para aqueles que procuram uma forma segura de poupança, com um \
            retorno estável e previsível, sem expor o capital a grandes oscilações de \
            mercado.\n\nSe tiveres mais perguntas sobre este produto ou outros da \
            Fidelidade, estou à disposição para ajudar!'

q1 = eval_chat(client2, assistant2.id, question, message2, instructions = None)

In [27]:
print(q1)

### Similarity Score: 6/10

### Summary of Improvements:

1. **Accuracy**:
   - The proposed answer inaccurately identifies the **Fidelidade My Savings** as the product in question. The correct product is **Fidelidade Savings**, which includes options tailored to different risk profiles: **Seguro** (low risk), **Proteção** (moderate risk with some exposure), and **Dinâmico** (higher risk). It should have specified that the choice of option greatly impacts the risk level.

2. **Completeness**:
   - The proposed answer is incomplete as it provides a generalization about a "low to moderate" risk profile without detailing the specific options available within the **Fidelidade Savings** product. The answer could be improved by mentioning these options and their associated risk levels clearly.

3. **Relevance**:
   - While the general idea of a "low to moderate" risk profile is somewhat relevant, the answer fails to cover the diversity within the **Fidelidade Savings** options. It should acc